In [35]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv('FMPKEY')
stocks = ['FTEC', 'SPYG', 'MTUM', 'SGDJ', 'FLSW', 'SPSM', 'EFV', 'USMV', 'IVE',
          'EELV', 'EFG', 'IFRA', 'EOG', 'MET', 'DBO', 'DBE', 'USCI', 'SCHX', 'SCHA', 'SPDW']
# stocks = ['EELV','EFG','EFV','FLSW','FTEC','IFRA','IVE','MTUM','SGDJ','SPSM','SPYG','USMV']
acct = 7762.89  # Account balance
risk = .2  # Risk tolerance
RF = .0282  # risk free rate 2.82% per annum
number_of_portfolios = 100000 # Number of portfolios to simulate

empresas = {}

# Get all prices into a dataframe
for stock in stocks:
    prices = requests.get(
        f'https://financialmodelingprep.com/api/v3/historical-price-full/{stock}?serietype=line&apikey={api_key}').json()
    prices_df = pd.DataFrame(prices['historical'])

    # Ensure the data is sorted by date in descending order (most recent first)
    prices_df['date'] = pd.to_datetime(prices_df['date'])
    prices_df = prices_df.sort_values('date', ascending=True)

    # Select the most recent 900 days
    recent_prices = prices_df.tail(900).set_index('date')['close']
    empresas[stock] = recent_prices


# Concatenate all stock close prices into a single dataframe
portfolio = pd.concat(empresas, axis=1)

# Calculate daily returns
return_stocks = portfolio.pct_change().dropna()

return_stocks

# Calculate mean returns and covariances of all assets
portfolio_returns = []
portfolio_risk = []
sharpe_ratio_port = []
portfolio_weights = []

for portfolio in range(number_of_portfolios):
    weights = np.random.random_sample((len(stocks)))
    weights = weights / np.sum(weights)
    annualize_return = np.sum((return_stocks.mean() * weights) * 252) - RF
    portfolio_returns.append(annualize_return)

    matrix_covariance_portfolio = (return_stocks.cov())*252
    portfolio_variance = np.dot(weights.T, np.dot(
        matrix_covariance_portfolio, weights))
    portfolio_standard_deviation = np.sqrt(portfolio_variance)
    portfolio_risk.append(portfolio_standard_deviation)

    sharpe_ratio = ((annualize_return - RF)/portfolio_standard_deviation)
    sharpe_ratio_port.append(sharpe_ratio)

    portfolio_weights.append(weights)

# Convert lists to arrays outside the loop
portfolio_risk = np.array(portfolio_risk)
portfolio_returns = np.array(portfolio_returns)
sharpe_ratio_port = np.array(sharpe_ratio_port)

# Create a dataframe with the returns and risk values of each portfolio
plt.figure(figsize=(10, 5))
plt.scatter(portfolio_risk, portfolio_returns,
            c=portfolio_returns / portfolio_risk)
plt.xlabel('volatility')
plt.ylabel('returns')
plt.colorbar(label='Sharpe ratio')

porfolio_metrics = [portfolio_returns, portfolio_risk,
                    sharpe_ratio_port, portfolio_weights]

# from Python list we create a Pandas DataFrame
portfolio_dfs = pd.DataFrame(porfolio_metrics)
portfolio_dfs = portfolio_dfs.T

# Rename the columns:
portfolio_dfs.columns = ['Port Returns',
                         'Port Risk', 'Sharpe Ratio', 'Portfolio Weights']

# convert from object to float the first three columns.
for col in ['Port Returns', 'Port Risk', 'Sharpe Ratio']:
    portfolio_dfs[col] = portfolio_dfs[col].astype(float)
portfolio_dfs

# portfolio with the highest Sharpe Ratio
highest_sharpe_port = portfolio_dfs.iloc[portfolio_dfs['Sharpe Ratio'].idxmax(
)]

# Extracting the weights
highest_sharpe_weights = highest_sharpe_port['Portfolio Weights']

# Printing the asset symbols along with their respective weights

# Extracting and printing the metrics
print(f"Portfolio Metrics for the Highest Sharpe Ratio Portfolio:")
print(f"Number of porfolios analyzed: {number_of_portfolios}")
print(f"Returns: {highest_sharpe_port['Port Returns']*100:.2f}%")
print(
    f"Risk (Standard Deviation): {highest_sharpe_port['Port Risk']*100:.2f}%")
print(f"Sharpe Ratio: {highest_sharpe_port['Sharpe Ratio']:.2f}")
print(f"")
print("Asset Weights in the Portfolio with the Highest Sharpe Ratio:")
# BEGIN: 8f7e6d4g3hj2
sorted_stocks = sorted(zip(stocks, highest_sharpe_weights),
                       key=lambda x: x[1], reverse=True)

print("Stocks sorted by highest Sharpe weights in descending order:")
for stock, weight in sorted_stocks:
    print(f"{stock}: {weight*100:.2f}%, ${weight*acct:.2f}")

# Calculate the correlation matrix
correlation_matrix = return_stocks.corr()

# Visualize the correlation matrix using a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", linewidths=0.5)
plt.title("Correlation Matrix of Asset Returns")
plt.show()